In [5]:
COST_INDEX = 0.07
SELL_TAX = 0.036
LOCATION = 60003760
REGION = 10000002
DURATION_MODIFIER = (1 - 0.32) * (1 - 0.15)
BP_LIST = """Capital Quantum Microprocessor Blueprint	10	0	-1	Advanced Capital Construction Components
Capital Crystalline Carbonide Armor Plate Blueprint	0	0	-1	Advanced Capital Construction Components
Capital Compact Pb-Acid Cap Battery Blueprint	0	0	-1	Capacitor Battery
Cap Booster 150 Blueprint	10	20	-1	Capacitor Booster Charge
Skirmish Command Burst I Blueprint	0	0	-1	Command Burst
Shield Command Burst I Blueprint	0	0	-1	Command Burst
Sustained Shield Emitter Blueprint	10	20	-1	Construction Components
Caldari Control Tower Small Blueprint	10	16	5	Control Tower
Caldari Control Tower Small Blueprint	10	16	5	Control Tower
Minmatar Control Tower Small Blueprint	0	0	-1	Control Tower
Amarr Control Tower Small Blueprint	0	0	-1	Control Tower
Gallente Control Tower Small Blueprint	0	0	-1	Control Tower
Corporate Hangar Array Blueprint	7	0	-1	Corporate Hangar Array
Capital Gremlin Compact Energy Neutralizer Blueprint	0	0	-1	Energy Neutralizer
Capital Infectious Scoped Energy Neutralizer Blueprint	0	0	-1	Energy Neutralizer
Warfare Computation Core Blueprint	10	20	-1	Hybrid Tech Components
Optimized Nano-Engines Blueprint	10	20	-1	Hybrid Tech Components
Metallofullerene Plating Blueprint	0	0	-1	Hybrid Tech Components
Superconducting Ladar Amplifier Blueprint	10	20	-1	Hybrid Tech Components
Reconfigured Subspace Calibrator Blueprint	10	0	-1	Hybrid Tech Components
Light Hull Maintenance Bot I Blueprint	10	20	-1	Logistic Drone
EMP M Blueprint	0	0	-1	Projectile Ammo
50000MN Cold-Gas Enduring Microwarpdrive Blueprint	0	0	-1	Propulsion Module
10000MN Monopropellant Enduring Afterburner Blueprint	0	0	-1	Propulsion Module
Capital Coaxial Compact Remote Armor Repairer Blueprint	0	0	-1	Remote Armor Repairer
Capital I-ax Enduring Remote Armor Repairer Blueprint	0	0	-1	Remote Armor Repairer
Large Thermal Armor Reinforcer I Blueprint	8	0	-1	Rig Armor
Small Processor Overclocking Unit I Blueprint	10	0	-1	Rig Core
Small Command Processor I Blueprint	0	0	-1	Rig Core
Small Energy Locus Coordinator I Blueprint	0	0	-1	Rig Energy Weapon
Small Hyperspatial Velocity Optimizer I Blueprint	10	0	-1	Rig Navigation
Small Low Friction Nozzle Joints I Blueprint	10	0	-1	Rig Navigation
Medium Low Friction Nozzle Joints I Blueprint	0	0	-1	Rig Navigation
Small Signal Focusing Kit I Blueprint	10	0	-1	Rig Scanning
Capital Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Large Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Scan Rangefinding Array I Blueprint	10	0	-1	Scanning Upgrade
Large Shield Extender I Blueprint	0	0	-1	Shield Extender
Ship Maintenance Array Blueprint	7	0	-1	Ship Maintenance Array
Caldari Shuttle Blueprint	7	10	-1	Shuttle
Ice Harvester I Blueprint	8	0	-1	Strip Miner
Standup Ladar ECM Script Blueprint	10	20	-1	Structure ECM script
Optimal Range Disruption Script Blueprint	8	0	-1	Tracking Disruption Script
Nova XL Torpedo Blueprint	8	16	-1	XL Torpedo"""

In [6]:
from src.util import connect_to_db

conn = await connect_to_db()

bp_info = await conn.fetch('SELECT * FROM sde.blueprints')
bp_materials = await conn.fetch("SELECT * FROM sde.blueprint_materials WHERE activity = 'manufacturing'")
bp_products = await conn.fetch("SELECT * FROM sde.blueprint_products WHERE activity = 'manufacturing'")
bp_products = {bp['blueprint_type_id']: {'type_id':bp['product_type_id'], 'quantity': bp['quantity']} for bp in bp_products}

type_ids = dict(await conn.fetch('SELECT type_id, name FROM sde.type_ids'))
type_names = dict(await conn.fetch('SELECT name, type_id FROM sde.type_ids'))
type_published = dict(await conn.fetch('SELECT type_id, published FROM sde.type_ids'))

market_buy = dict(await conn.fetch(f'SELECT type_id, buy_max FROM market.aggregates WHERE location_id = {LOCATION}'))
market_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE location_id = {LOCATION}'))
market_region_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE region_id = {REGION}'))

blueprints = {}
for bp in bp_info:
    if type_published[bp['blueprint_type_id']] == False:
        continue
    if bp['blueprint_type_id'] not in bp_products:
        continue
    blueprints[bp['blueprint_type_id']] = {
        'name': type_ids[bp['blueprint_type_id']],
        'manufacture_time': bp['manufacturing_time'],
        'materials': [],
        'product': bp_products[bp['blueprint_type_id']],
    }
for bp_mat in bp_materials:
    if bp_mat['blueprint_type_id'] not in blueprints:
        continue
    blueprints[bp_mat['blueprint_type_id']]['materials'].append({
        'type_id': bp_mat['material_type_id'],
        'quantity': bp_mat['quantity'],
    })

In [7]:
import math
from decimal import Decimal

SELL_TAX = Decimal(SELL_TAX)
COST_INDEX = Decimal(COST_INDEX)

sell_price = {}
buy_price = {}
profit = {}
profit_per_day = {}
profitable_price = {}
bp_price = {}

owned_mat_eff = {}
owned_time_eff = {}
for bp_line in BP_LIST.split("\n"):
    bp_name, bp_mat_eff, bp_time_eff, _, _ = bp_line.split("\t")
    owned_mat_eff[type_names[bp_name]] = int(bp_mat_eff) / 100
    owned_time_eff[type_names[bp_name]] = int(bp_time_eff) / 100

for bp_id, bp in blueprints.items():
    if bp['product']['type_id'] not in market_buy or market_buy[bp['product']['type_id']] is None:
        continue
    if bp['product']['type_id'] not in market_sell or market_sell[bp['product']['type_id']] is None:
        continue
    
    bp_price[bp_id] = market_region_sell[bp_id] if bp_id in market_region_sell else 0
    bp_price[bp_id] = bp_price[bp_id] if bp_price[bp_id] is not None else 0
    
    buy_price[bp_id] = 0
    cant_buy = False
    for material in bp['materials']:
        if material['type_id'] not in market_sell or market_sell[material['type_id']] is None:
            cant_buy = True
            break
        buy_quantity = math.ceil(material['quantity'] * (1 - owned_mat_eff[bp_id] if bp_id in owned_mat_eff else 1))
        buy_price[bp_id] += market_sell[material['type_id']] * buy_quantity
        
    if cant_buy:
        continue
    
    profitable_price[bp_id] = buy_price[bp_id] / (1 - COST_INDEX) / (1 - SELL_TAX) / bp['product']['quantity']
    
    sell_price[bp_id] = market_buy[bp['product']['type_id']] * bp['product']['quantity']
    # sell_price[bp_id] = market_sell[bp['product']['type_id']] * bp['product']['quantity']
    profit[bp_id] = sell_price[bp_id] - buy_price[bp_id] - sell_price[bp_id] * SELL_TAX - buy_price[bp_id] * COST_INDEX
    profit_per_day[bp_id] = profit[bp_id] * Decimal(86400 / (bp['manufacture_time'] * DURATION_MODIFIER * (1 - owned_time_eff[bp_id] if bp_id in owned_time_eff else 1)))

print(f"{'Name':<64} {'Profit per day':>16} {'Profit':>16} {'Buy price':>16} {'Sell price':>16} {'Runs/BP':>8} {'Days/BP':>8}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_price[bp_id] == 0:
        continue
    if bp_id in owned_mat_eff:
        continue
    print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {bp_price[bp_id] / profit[bp_id]:8.2f} {bp_price[bp_id] / profit_per_day[bp_id]:8.2f}")
    

Name                                                               Profit per day           Profit        Buy price       Sell price  Runs/BP  Days/BP
Magnetometric-FTL Interlink Communicator Blueprint                 280,968,486.63    30,074,034.31   126,944,267.00   172,100,000.00    66.40     7.11
Radar-FTL Interlink Communicator Blueprint                          91,446,424.35     9,788,154.31   131,668,267.00   156,300,000.00   219.65    23.51
'Azmaru' Electromagnetic Disruptive Lance Blueprint                 50,284,177.16    24,220,212.00   231,968,400.00   282,600,000.00    21.45    10.33
Personal Hangar Array Blueprint                                     14,688,142.70     1,179,131.46    16,925,708.92    20,010,000.00   254.42    20.42
Ogre SD-900 Blueprint                                               10,778,617.78        64,896.26        97,012.84       175,000.00   107.86     0.65
Medium Industrial Core II Blueprint                                 10,608,560.02     2,491,02

In [8]:
print(f"{'Name':<64} {'Profit / day':>16} {'Profit / run':>16} {'Buy price':>16} {'Sell price':>16} {'Profitable price':>16}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_id in owned_mat_eff:
        print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {profitable_price[bp_id]:16,.2f}")

Name                                                                 Profit / day     Profit / run        Buy price       Sell price Profitable price
Capital Core Defense Operational Solidifier I Blueprint             21,397,182.23       858,859.12    21,423,384.00    24,670,000.00    23,896,158.48
10000MN Monopropellant Enduring Afterburner Blueprint               13,478,453.98     3,246,061.00    60,977,700.00    71,050,000.00    68,015,995.18
Shield Command Burst I Blueprint                                     6,478,486.41       156,023.55       562,588.46       786,300.00       627,524.72
Medium Low Friction Nozzle Joints I Blueprint                        3,045,963.74        61,130.80       667,400.00       804,200.00       744,434.03
Small Energy Locus Coordinator I Blueprint                           2,817,740.18        28,275.24        63,668.00       100,000.00        71,016.82
Small Command Processor I Blueprint                                  2,771,979.07        27,816.04  